In [1]:
## Megan getting errors with Yoni script src folder 
# copy and pasting functions here 

# script goal - use mediapipe world outputs to estimate depth in meters 

In [11]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
#from src.analysis_funs import get_landmark_pairs, get_all_unique_pairs
def get_all_unique_pairs(df):
    df_pairs = df.merge(df, on='frame', how='left')
    df_pairs = df_pairs[df_pairs['label_x'] != df_pairs['label_y']]
    def make_id(x, y, i):
        return str(i) + "-" +  str(max(x,y)) + "-" + str(min(x,y))
    df_pairs["row_id"] = df_pairs.apply(lambda row: make_id(row["subframe_id_x"],
                                                                row["subframe_id_y"],
                                                                row["frame"]), axis=1)
    return df_pairs.drop_duplicates(subset="row_id", keep="first")


def get_landmark_pairs(landmark_df, label_x, label_y):
    tmp = landmark_df.merge(landmark_df, on='frame', how='left')
    return tmp[(tmp.label_x == label_x) & (tmp.label_y == label_y)].copy()

In [4]:
output_path = r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\test_velocity_01302025\hc_video_output\MM_HC_test_distance\09-12-2024\000_raw_pose_data'
file_name = 'mari_mpipe_gait_vertical_left_df.csv'

mpipe_path = r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\test_velocity_01302025\hc_video_output\MM_HC_test_distance\09-12-2024\000_raw_pose_data\2024_09_12_02_41_PWS_1_gait_vertical_left_mediapipe_world.csv'

mari_path = r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\test_velocity_01302025\hc_video_output\MM_HC_test_distance\09-12-2024\000_raw_pose_data\2024_09_12_02_41_PWS_1_gait_vertical_left_marigold.csv'

In [5]:
mpipe = pd.read_csv(mpipe_path)

mpipe = mpipe.rename(columns={'Unnamed: 0': 'subframe_id'})
mpipe_filt = mpipe[mpipe['label'].notnull()]

mpipe_pairs = get_all_unique_pairs(mpipe_filt)
mpipe_pairs.head()

,subframe_id_x,X_x,Y_x,Z_x,vis_x,pres_x,frame,label_x,subframe_id_y,X_y,Y_y,Z_y,vis_y,pres_y,label_y,row_id
1,0,-0.003591,-0.670364,0.093021,0.999953,0.0,0,nose,1,-0.022470,-0.693933,0.052621,0.999938,0.0,left_eye_inner,0-1-0
2,0,-0.003591,-0.670364,0.093021,0.999953,0.0,0,nose,2,-0.022258,-0.694207,0.053130,0.999939,0.0,left_eye,0-2-0
3,0,-0.003591,-0.670364,0.093021,0.999953,0.0,0,nose,3,-0.022580,-0.694672,0.053112,0.999941,0.0,left_eye_outer,0-3-0
4,0,-0.003591,-0.670364,0.093021,0.999953,0.0,0,nose,4,-0.010399,-0.694235,0.056104,0.999929,0.0,right_eye_inner,0-4-0
5,0,-0.003591,-0.670364,0.093021,0.999953,0.0,0,nose,5,-0.010047,-0.695350,0.055322,0.999928,0.0,right_eye,0-5-0


In [6]:
mari = pd.read_csv(mari_path)

mari = mari.rename(columns={'Unnamed: 0': 'subframe_id'})
mari_filt = mari[mari['label'].notnull()]

mari_pairs = get_all_unique_pairs(mari_filt)
mari_pairs.head()

,subframe_id_x,frame,label_x,depth_est_x,subframe_id_y,label_y,depth_est_y,row_id
1,0,0,nose,0.34744,1,left_eye_inner,0.351213,0-1-0
2,0,0,nose,0.34744,2,left_eye,0.352838,0-2-0
3,0,0,nose,0.34744,3,left_eye_outer,0.354305,0-3-0
4,0,0,nose,0.34744,4,right_eye_inner,0.355535,0-4-0
5,0,0,nose,0.34744,5,right_eye,0.357866,0-5-0


In [7]:
mari_mpipe = mari_pairs.merge(mpipe_pairs, on=['row_id'])
print(f"mari_pipe type: {type(mari_mpipe)}")
mari_mpipe.head()

mari_pipe type: <class 'pandas.core.frame.DataFrame'>


,subframe_id_x_x,frame_x,label_x_x,depth_est_x,subframe_id_y_x,label_y_x,depth_est_y,row_id,subframe_id_x_y,X_x,...,pres_x,frame_y,label_x_y,subframe_id_y_y,X_y,Y_y,Z_y,vis_y,pres_y,label_y_y
0,0,0,nose,0.34744,1,left_eye_inner,0.351213,0-1-0,0,-0.003591,...,0.0,0,nose,1,-0.022470,-0.693933,0.052621,0.999938,0.0,left_eye_inner
1,0,0,nose,0.34744,2,left_eye,0.352838,0-2-0,0,-0.003591,...,0.0,0,nose,2,-0.022258,-0.694207,0.053130,0.999939,0.0,left_eye
2,0,0,nose,0.34744,3,left_eye_outer,0.354305,0-3-0,0,-0.003591,...,0.0,0,nose,3,-0.022580,-0.694672,0.053112,0.999941,0.0,left_eye_outer
3,0,0,nose,0.34744,4,right_eye_inner,0.355535,0-4-0,0,-0.003591,...,0.0,0,nose,4,-0.010399,-0.694235,0.056104,0.999929,0.0,right_eye_inner
4,0,0,nose,0.34744,5,right_eye,0.357866,0-5-0,0,-0.003591,...,0.0,0,nose,5,-0.010047,-0.695350,0.055322,0.999928,0.0,right_eye


In [8]:
def check_row(labxx,labyx,labxy,labyy):
    return (labxx == labxy) and (labyx == labyy)


mari_mpipe["label_agreement"] = mari_mpipe.apply(lambda row: check_row(row["label_x_x"],
                                                                       row["label_y_x"],
                                                                       row["label_x_y"],
                                                                       row["label_y_y"]),
                                                 axis=1)
mari_mpipe.head()

,subframe_id_x_x,frame_x,label_x_x,depth_est_x,subframe_id_y_x,label_y_x,depth_est_y,row_id,subframe_id_x_y,X_x,...,frame_y,label_x_y,subframe_id_y_y,X_y,Y_y,Z_y,vis_y,pres_y,label_y_y,label_agreement
0,0,0,nose,0.34744,1,left_eye_inner,0.351213,0-1-0,0,-0.003591,...,0,nose,1,-0.022470,-0.693933,0.052621,0.999938,0.0,left_eye_inner,True
1,0,0,nose,0.34744,2,left_eye,0.352838,0-2-0,0,-0.003591,...,0,nose,2,-0.022258,-0.694207,0.053130,0.999939,0.0,left_eye,True
2,0,0,nose,0.34744,3,left_eye_outer,0.354305,0-3-0,0,-0.003591,...,0,nose,3,-0.022580,-0.694672,0.053112,0.999941,0.0,left_eye_outer,True
3,0,0,nose,0.34744,4,right_eye_inner,0.355535,0-4-0,0,-0.003591,...,0,nose,4,-0.010399,-0.694235,0.056104,0.999929,0.0,right_eye_inner,True
4,0,0,nose,0.34744,5,right_eye,0.357866,0-5-0,0,-0.003591,...,0,nose,5,-0.010047,-0.695350,0.055322,0.999928,0.0,right_eye,True


In [9]:
mari_mpipe["depth_to_m"] = np.abs(mari_mpipe["Z_x"] - mari_mpipe["Z_y"]) / np.abs(mari_mpipe["depth_est_x"] - mari_mpipe["depth_est_y"])
mari_mpipe.head()

,subframe_id_x_x,frame_x,label_x_x,depth_est_x,subframe_id_y_x,label_y_x,depth_est_y,row_id,subframe_id_x_y,X_x,...,label_x_y,subframe_id_y_y,X_y,Y_y,Z_y,vis_y,pres_y,label_y_y,label_agreement,depth_to_m
0,0,0,nose,0.34744,1,left_eye_inner,0.351213,0-1-0,0,-0.003591,...,nose,1,-0.022470,-0.693933,0.052621,0.999938,0.0,left_eye_inner,True,10.707573
1,0,0,nose,0.34744,2,left_eye,0.352838,0-2-0,0,-0.003591,...,nose,2,-0.022258,-0.694207,0.053130,0.999939,0.0,left_eye,True,7.389272
2,0,0,nose,0.34744,3,left_eye_outer,0.354305,0-3-0,0,-0.003591,...,nose,3,-0.022580,-0.694672,0.053112,0.999941,0.0,left_eye_outer,True,5.812797
3,0,0,nose,0.34744,4,right_eye_inner,0.355535,0-4-0,0,-0.003591,...,nose,4,-0.010399,-0.694235,0.056104,0.999929,0.0,right_eye_inner,True,4.560005
4,0,0,nose,0.34744,5,right_eye,0.357866,0-5-0,0,-0.003591,...,nose,5,-0.010047,-0.695350,0.055322,0.999928,0.0,right_eye,True,3.615665


In [12]:
mari_mpipe.to_csv(os.path.join(output_path, file_name)) 